1. Detect topics from messages send from the top 3 country in the dataset
In the previous file, I identified top 3 countries(Singapore, India, and United States)
What are common topics of their messages

In [21]:
import pandas as pd
import nltk 
from nltk.probability import FreqDist
import gensim
import re


In [30]:
messages = pd.read_csv("processed_cleaned_sms.csv")
messages.head()

,id,Message,length,country,Date,year,month,Message_clean,Message_tokenized,Message_tokenized_nostop,Message_lemmatized
0,10120,Bugis oso near wat...,21,Singapore,2003/4,2003,4,bugis oso near wat,"['bugis', 'oso', 'near', 'wat']","['bugis', 'oso', 'near', 'wat']","['bugis', 'oso', 'near', 'wat']"
1,10121,"Go until jurong point, crazy.. Available only ...",111,Singapore,2003/4,2003,4,go until jurong point crazy available only in ...,"['go', 'until', 'jurong', 'point', 'crazy', 'a...","['go', 'jurong', 'point', 'crazy', 'available'...","['go', 'jurong', 'point', 'crazy', 'available'..."
2,10122,I dunno until when... Lets go learn pilates...,46,Singapore,2003/4,2003,4,i dunno until when lets go learn pilates,"['i', 'dunno', 'until', 'when', 'lets', 'go', ...","['dunno', 'lets', 'go', 'learn', 'pilates']","['dunno', 'let', 'go', 'learn', 'pilate']"
3,10123,Den only weekdays got special price... Haiz......,140,Singapore,2003/4,2003,4,den only weekdays got special price haiz cant ...,"['den', 'only', 'weekdays', 'got', 'special', ...","['den', 'weekdays', 'got', 'special', 'price',...","['den', 'weekday', 'get', 'special', 'price', ..."
4,10124,Meet after lunch la...,22,Singapore,2003/4,2003,4,meet after lunch la,"['meet', 'after', 'lunch', 'la']","['meet', 'lunch', 'la']","['meet', 'lunch', 'la']"


In [31]:
sg_messages = messages[messages['country'] == 'Singapore']
india_messages = messages[messages['country'] == 'India']
us_messages = messages[messages['country'] == 'United States']


In [32]:
import ast 

In [33]:
sg_messages["Message_lemmatized"] = sg_messages["Message_lemmatized"].apply(ast.literal_eval)
all_words_sg = [word for message in sg_messages["Message_lemmatized"] for word in message]
fdist_sg = FreqDist(all_words_sg)
fdist_sg.most_common(15)

C:\Users\exol1\AppData\Local\Temp\ipykernel_15724\555868546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sg_messages["Message_lemmatized"] = sg_messages["Message_lemmatized"].apply(ast.literal_eval)


[('u', 8284),
 ('haha', 6631),
 ('go', 4476),
 ('get', 3097),
 ('lol', 2818),
 ('ok', 1813),
 ('hahaha', 1752),
 ('late', 1707),
 ('time', 1629),
 ('okay', 1609),
 ('le', 1458),
 ('think', 1448),
 ('come', 1372),
 ('meet', 1366),
 ('one', 1264)]

In [34]:
india_messages["Message_lemmatized"] = india_messages["Message_lemmatized"].apply(ast.literal_eval)
all_words_india = [word for message in india_messages["Message_lemmatized"] for word in message]
fdist_india = FreqDist(all_words_india)
fdist_india.most_common(15)

C:\Users\exol1\AppData\Local\Temp\ipykernel_15724\1628783136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  india_messages["Message_lemmatized"] = india_messages["Message_lemmatized"].apply(ast.literal_eval)


[('u', 1191),
 ('k', 404),
 ('come', 393),
 ('2', 338),
 ('ur', 324),
 ('call', 323),
 ('get', 302),
 ('ok', 301),
 ('go', 300),
 ('r', 295),
 ('hey', 247),
 ('n', 242),
 ('``', 240),
 ('day', 237),
 ('good', 232)]

In [35]:
us_messages["Message_lemmatized"] = us_messages["Message_lemmatized"].apply(ast.literal_eval)
all_words_us = [word for message in us_messages["Message_lemmatized"] for word in message]
fdist_us = FreqDist(all_words_us)
fdist_us.most_common(15)

C:\Users\exol1\AppData\Local\Temp\ipykernel_15724\2696671929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_messages["Message_lemmatized"] = us_messages["Message_lemmatized"].apply(ast.literal_eval)


[('get', 602),
 ('know', 340),
 ('u', 336),
 ('go', 315),
 ('lol', 278),
 ('like', 262),
 ('thanks', 256),
 ('want', 252),
 ('come', 232),
 ('yeah', 207),
 ('time', 205),
 ('hi', 201),
 ('think', 192),
 ('see', 189),
 ('love', 182)]

In [39]:
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from sklearn.model_selection import GridSearchCV

In [ ]:
sg_dict = Dictionary(sg_messages["Message_lemmatized"])
sg_bow_corpus = [sg_dict.doc2bow(message) for message in sg_messages["Message_lemmatized"]]
sg_lda_model = LdaModel(corpus=sg_bow_corpus, id2word=sg_dict, num_topics=5, random_state=2) 
for topic in sg_lda_model.print_topics():
    print(topic)

(0, '0.100*"haha" + 0.027*"go" + 0.020*"u" + 0.018*"think" + 0.014*"want" + 0.010*"p" + 0.010*"dun" + 0.010*"x" + 0.010*"lol" + 0.009*"get"')
(1, '0.046*"okay" + 0.032*"le" + 0.018*"ohh" + 0.018*"oh" + 0.016*"haha" + 0.012*"get" + 0.011*"p" + 0.010*"watch" + 0.010*"go" + 0.009*"yeah"')
(2, '0.078*"hahaha" + 0.052*"haha" + 0.036*"yeah" + 0.027*"get" + 0.015*"one" + 0.013*"go" + 0.012*"la" + 0.010*"omg" + 0.010*"dun" + 0.010*"play"')
(3, '0.031*"go" + 0.023*"ok" + 0.019*"u" + 0.019*"meet" + 0.017*"yeap" + 0.014*"come" + 0.014*"wan" + 0.013*"home" + 0.013*"leave" + 0.013*"na"')
(4, '0.069*"lol" + 0.027*"haha" + 0.014*"sorry" + 0.011*"like" + 0.011*"okay" + 0.009*"sure" + 0.009*"fine" + 0.009*"go" + 0.009*"alot" + 0.009*"yay"')


In [41]:
sg_perplexity = sg_lda_model.log_perplexity(sg_bow_corpus)
sg_perplexity
import math 

math.exp(sg_perplexity)

0.00036693565976519407

In [43]:
import pyLDAvis
import pyLDAvis.gensim


In [44]:
vis_sg = pyLDAvis.gensim.prepare(sg_lda_model, sg_bow_corpus, sg_dict)
pyLDAvis.display(vis_sg)

In [45]:
india_dict = Dictionary(india_messages["Message_lemmatized"])
india_bow_corpus = [india_dict.doc2bow(message) for message in india_messages["Message_lemmatized"]]
india_lda_model = LdaModel(corpus=india_bow_corpus, id2word=india_dict, num_topics=5, random_state=2) 
for topic in india_lda_model.print_topics():
    print(topic)

(0, '0.053*"u" + 0.031*"ok" + 0.012*"r" + 0.010*"come" + 0.008*"2" + 0.007*"get" + 0.007*"ur" + 0.007*"send" + 0.007*"4" + 0.007*"say"')
(1, '0.018*"u" + 0.017*"hi" + 0.013*"go" + 0.013*"happy" + 0.012*"come" + 0.011*"day" + 0.011*"ur" + 0.010*"get" + 0.009*"reach" + 0.008*"please"')
(2, '0.028*"come" + 0.014*"go" + 0.014*"yes" + 0.009*"time" + 0.009*"back" + 0.007*"sleep" + 0.007*"work" + 0.006*"dinner" + 0.006*"yesterday" + 0.006*"u"')
(3, '0.018*"good" + 0.016*"sure" + 0.016*"k" + 0.013*"oh" + 0.011*"\'\'" + 0.011*"hai" + 0.010*"nice" + 0.009*"class" + 0.008*"today" + 0.008*"yup"')
(4, '0.039*"call" + 0.025*"meet" + 0.018*"hey" + 0.014*"see" + 0.013*"sorry" + 0.013*"late" + 0.009*"let" + 0.009*"soon" + 0.009*"know" + 0.008*"free"')


In [46]:
india_perplexity = india_lda_model.log_perplexity(india_bow_corpus)
india_perplexity

math.exp(india_perplexity)

0.00022675700316183422

In [47]:
vis_india = pyLDAvis.gensim.prepare(india_lda_model, india_bow_corpus, india_dict)
pyLDAvis.display(vis_india)

In [48]:
us_dict = Dictionary(us_messages["Message_lemmatized"])
us_bow_corpus = [us_dict.doc2bow(message) for message in us_messages["Message_lemmatized"]]
us_lda_model = LdaModel(corpus=us_bow_corpus, id2word=us_dict, num_topics=5, random_state=2) 
for topic in us_lda_model.print_topics():
    print(topic)

(0, '0.016*"come" + 0.015*"haha" + 0.015*"xp" + 0.013*"get" + 0.013*"``" + 0.012*"ok" + 0.012*"alright" + 0.010*"love" + 0.010*"really" + 0.009*"meet"')
(1, '0.017*"sorry" + 0.016*"p" + 0.014*"post" + 0.010*"like" + 0.009*"yeah" + 0.009*"lol" + 0.009*"great" + 0.008*"take" + 0.007*"``" + 0.007*"know"')
(2, '0.020*"go" + 0.019*"know" + 0.017*"love" + 0.015*"lol" + 0.015*"get" + 0.012*"p" + 0.011*"u" + 0.010*"time" + 0.010*"need" + 0.010*"much"')
(3, '0.027*"lol" + 0.015*"hi" + 0.013*"oh" + 0.013*"well" + 0.013*"like" + 0.013*"one" + 0.011*"name" + 0.010*"know" + 0.009*"think" + 0.008*"yeah"')
(4, '0.036*"get" + 0.017*"na" + 0.015*"u" + 0.014*"want" + 0.014*"thanks" + 0.011*"gon" + 0.011*"come" + 0.011*"yeah" + 0.009*"call" + 0.008*"make"')


In [49]:
us_perplexity = us_lda_model.log_perplexity(us_bow_corpus)
us_perplexity

math.exp(us_perplexity)

0.0003516930765523271

In [50]:
vis_us = pyLDAvis.gensim.prepare(us_lda_model, us_bow_corpus, us_dict)
pyLDAvis.display(vis_us)